In [12]:
import os
import sys
import rdkit
import svgutils.transform as sg

from jazzy import core, visualisation, helpers
# from cairosvg import svg2png

In [13]:
# Import config
opt_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'optimisation'))
sys.path.insert(0, opt_path)
import config

# Open the compressed data
data_path = os.path.abspath(os.path.join(os.getcwd(), '..', config.DATA_PATH))

In [14]:
# Get all molecules in folder
cdk2_path = os.path.join(data_path, "cdk2_analysis")
files = os.listdir(cdk2_path)
files = [f for f in files if f.endswith("sdf") or f.endswith("mol")]
files

['20Z.sdf', '26Z.sdf', 'X02.sdf', 'X35.sdf', 'X36.sdf', 'X44.sdf']

In [15]:
def atomistic_strength_from_file(file_path):
    """Accepts a file path to an SDF/MOL file and generates an HB strength visualisation.
    """
    rdkit_mol = rdkit.Chem.MolFromMolFile(file_path)
    rdkit_mol = rdkit.Chem.AddHs(rdkit_mol, addCoords=True)
    kallisto_mol = core.kallisto_molecule_from_rdkit_molecule(rdkit_mol)
    atoms_and_nbrs = core.get_covalent_atom_idxs(rdkit_mol)
    charges = core.get_charges_from_kallisto_molecule(kallisto_mol, charge=0)
    atomic_map = core.calculate_polar_strength_map(rdkit_mol, kallisto_mol, atoms_and_nbrs, charges)
    img_text = visualisation.depict_strengths(rdkit_mol, 
                                            atomic_map, 
                                            fig_size=(500, 500),
                                            flatten_molecule=True, 
                                            highlight_atoms=True, 
                                            ignore_sdc=True, 
                                            ignore_sdx=False,
                                            ignore_sa=True,
                                            sdc_threshold=0.0, 
                                            sdx_threshold=0.0,
                                            sa_threshold=0.0,
                                            rounding_digits=2)

    # Add ligand name as a title
    img_text = img_text.replace('svg:','')
    fig = sg.fromstring(img_text)
    label = sg.TextElement(250, 15, file_path.split("/")[-1], size=14, 
                        font='sans-serif', anchor='middle', color='#000000')
    fig.append(label)
    img_text = fig.to_str().decode("utf-8")
    return img_text

In [16]:
# Build strength visualisations
strengths = list()
for f in files:
    file_path = os.path.join(cdk2_path, f)
    strengths.append(atomistic_strength_from_file(file_path))

In [17]:
class HorizontalDisplay:
    """
    Accepts a list of SVGs, concatenates them, and returns a horizonal rendering.
    """
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        concat_svgs = ''.join(self.args[0])
        template = '<div style="">{}</div>'
        return template.format(concat_svgs)

In [18]:
HorizontalDisplay(strengths)

In [19]:
def mol_vector_from_file(file_path):
    """Accepts a file path to an SDF/MOL file and generates a mol vector.
    """
    rdkit_mol = rdkit.Chem.MolFromMolFile(file_path)
    rdkit_mol = rdkit.Chem.AddHs(rdkit_mol, addCoords=True)
    kallisto_mol = core.kallisto_molecule_from_rdkit_molecule(rdkit_mol)
    atoms_and_nbrs = core.get_covalent_atom_idxs(rdkit_mol)
    charges = core.get_charges_from_kallisto_molecule(kallisto_mol, charge=0)
    atomic_map = core.calculate_polar_strength_map(rdkit_mol, kallisto_mol, atoms_and_nbrs, charges)
    mol_vector = helpers.sum_atomic_map(atomic_map)
    mol_vector["name"] = file_path.split("/")[-1]
    return mol_vector

In [20]:
# Build molecular vectors
vects = list()
for f in files:
    file_path = os.path.join(cdk2_path, f)
    vects.append(mol_vector_from_file(file_path))
vects

[{'sdc': 7.3073, 'sdx': 4.6706, 'sa': 7.8464, 'name': '20Z.sdf'},
 {'sdc': 5.585, 'sdx': 6.2794, 'sa': 8.6611, 'name': '26Z.sdf'},
 {'sdc': 5.3399, 'sdx': 2.6102, 'sa': 4.4025, 'name': 'X02.sdf'},
 {'sdc': 5.489, 'sdx': 2.6952, 'sa': 5.0008, 'name': 'X35.sdf'},
 {'sdc': 5.6043, 'sdx': 2.5862, 'sa': 4.3664, 'name': 'X36.sdf'},
 {'sdc': 6.7521, 'sdx': 6.9577, 'sa': 10.8753, 'name': 'X44.sdf'}]

In [21]:
# Export renderings
# for f in files:
#     file_path = os.path.join(cdk2_path, f)
#     output_filepath = f.split(".")[0] + ".png"
#     svg2png(bytestring=atomistic_strength_from_file(file_path), write_to=output_filepath)
